# General Information

This is a graphing function for X,Y data plus scalars. The goal of this function is to provide a fast UI to quickly visualize data coming off of a machine in realtime / shortly after making the measurments to verify that all data collected is satisfactory. This IS NOT for visualizing large sums of data, as it calculates everything from the raw data in realtime rather than pulling the scalars from a dataframe.

Variables to change are located in .getuserinputs().

Everything else is calculated from the class assuming it has the data structure outlined in XY + Scalar Data Class

# Import packages

In [1]:
# system files
import os

# math
import numpy as np
import math

# plotting
import glob
import matplotlib
from matplotlib import cm
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
from matplotlib.ticker import FormatStrFormatter
from matplotlib.ticker import ScalarFormatter
import seaborn as sns
from matplotlib.ticker import LinearLocator

# UI
import tkinter as tk
from tkinter import ttk

# Files on Comp
import SPD_JV_Class

# XY + Scalar Data Class

A General class section has a few main functions to work with the plotter:

A) __init__(): defines variables stored in self, it has:

     - self.ID = ID of Class (e.g. JV)
     
     - self.filepath = full path to file
     - self.filefolder = full path to folder holding file
     - self.filename = name of file including extension (e.g. JVFile.csv)
     - self.classname = name of class made from file (e.g. JVFile)
     
     - self.loadedlist: all loaded variable names
             self.load (0=not analyzed, 1=analyzed)
     - self.vectorlist: all calculated vector names
              self.vetcor (0=not analyzed, 1=analyzed)
     - self.scalarlist: all calculated scalar names
              self.scalar (0=not analyzed, 1=analyzed)

     - self.Varaible: jsc --> Jsc
     - self.Name: jsc --> Short Circuit Current Density / Jsc (mA/cm²)
     - self.NameRev: opposite of Name
     - self.Label: jsc --> Jsc (mA/cm²)
     - self.Unit: jsc --> mA/cm²

B) loadfile(fullpathtofile): loads file, sets self.load = 1, calculates all values in self.loadedlist

C) createlists(): creates lists and vectors for management of class

D) calcvectors(): sets self.vector = 1, calculates all vectors in self.vectorlist

E) calcscalars(): sets self.scalar = 1, calculates all vectors in self.vectorlist

F) getparamvals(inputvallist): returns scalars for varaible inputs

# Setup XYGraphingWindow

This class uses inputs in 'getuserinputs' to manage workup without anything specific to the class.

In [2]:
class xyGraphingWindow:
    
    
    # Takes User Inputs
    def getuserinputs(self):
        
        # edit to be the class of file to call
        self.classname = SPD_JV_Class.JVCurve
        
        # edit to be path to the folder to analyze
        self.loadfolder = 'C:\\Users\\seand\\Documents\\Data\\JV\\'
        
        # edit to change which data to plot using indices, these will be more consistent across classes than varnames
        self.xindex = 0 # this corresponds to v for jv waves
        self.yindex = 1 # this corresponds to j for jv waves
        
        # edit to give the data the color scheme youd like as well as change font sizes to fit your screen
        self.cmap = cm.get_cmap('jet')
        self.fontsizelarge = 12
        self.fontsizesmall = 10
    
   
    # customize graphs based on what type of graph it is
    def customizegraphs(self,thisgraph):         
       
        # Add X and Y labels for all graphs
        thisgraph.set_ylabel(self.dummy.Label[self.xydict['y']], fontsize=self.fontsizesmall)
        thisgraph.set_xlabel(self.dummy.Label[self.xydict['x']], fontsize=self.fontsizesmall)
        
        # If class is JV then do these special things to the graph:
        if self.dummy.ID == 'JV':
            thisgraph.axhline(0, color='black')
            thisgraph.axvline(0, color='black')
            thisgraph.set_ylim(bottom = -5) 
            
            
    # builds UI
    def __init__(self, graphinguserinterface):
        
        #get user input variables for customization
        self.getuserinputs()
        
        # create dummy file of class, call .createlists on dummy class, pull x and y for plotting
        self.dummy = self.classname()  
        self.dummy.createlists()
        self.xydict = {'x':self.dummy.loadedlist[self.xindex],'y':self.dummy.loadedlist[self.yindex]}
    
        # move into folder, make list of txt files in it
        os.chdir(self.loadfolder)
        self.listoffiles = glob.glob('*csv')

        # create dictionary { filename : classfilename } where jvname is same name - .txt or .csv
        self.filestoclass = {}
        for file in self.listoffiles:
            classfilename = file.rsplit(".", 1)[0]
            self.filestoclass[file] = classfilename

        # make self.analyzed waves to hold waves to operate stats on and plot in figure #2. 
        self.analyzedwaves = []

        # define main window, get dimensions of screen (for my computer the previous values are w=1500 and h=800)
        self.window = graphinguserinterface        
        self.maxwidth = math.floor(self.window.winfo_screenwidth()/100)*100-25
        self.maxheight = math.floor(self.window.winfo_screenheight()/100)*100-25           
        
        # using screen resolution set the width of various features
        self.spacerwidth = self.maxwidth/100
        self.scrollwidth = self.maxwidth/6
        self.f12width = (self.maxwidth-self.scrollwidth-self.spacerwidth*3)/3            
        self.checkwidth = (self.maxwidth-self.scrollwidth-self.spacerwidth*(2+len(self.dummy.scalarlist)))/len(self.dummy.scalarlist)
        self.statswidth = (self.maxwidth-self.scrollwidth-self.spacerwidth*2)/10          
        self.svswidth = (self.maxwidth-self.scrollwidth-self.spacerwidth*2)/5         

        # using screen resolution set the height of various features
        self.spacerheight = self.maxheight/100 
        self.scrollheight = self.maxheight/1       
        self.f12height = (self.maxheight)/3.5         
        self.checkheight = (self.maxheight)/30       
        self.statsheight = (self.maxheight)/5     
        self.svsheight = (self.maxheight)/5          
          
        # create main window just less than the size of screen, make backgrouund white and set title
        self.window.geometry(str(self.maxwidth)+'x'+str(self.maxheight)+'+0+0')
        self.window.configure(bg='white')
        self.window.title('FRG '+self.dummy.ID+' Analysis')

        # create parent frame to hold self.framefiles and self.framebuttons
        self.listboxandbuttonsframe = tk.Frame(self.window)

        # create framefiles for list of txt files + scroll bar, pack the frame left and right, put on top in parent frame
        self.framefiles = tk.Frame(self.listboxandbuttonsframe)      
        self.varfiles = tk.StringVar(value=self.listoffiles)
        self.listbox = tk.Listbox(self.framefiles,
                                  listvariable=self.varfiles,
                                  selectmode="extended",
                                  bg='#FAFAFA')
        self.scrollbar = tk.Scrollbar(self.framefiles, orient="vertical")
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.BOTH)
        self.listbox.pack(side=tk.LEFT, expand=True, fill=tk.BOTH)
        self.listbox['yscrollcommand'] = self.scrollbar.set
        self.scrollbar['command'] = self.listbox.yview
        self.framefiles.pack(side=tk.TOP, expand =True, fill=tk.BOTH)
        
        # bind listbox info --> when item selected update plot #1 and colors
        self.listbox.bind('<<ListboxSelect>>', self.onselect)

        # create framebuttons1 (add/remove) above framebuttons2 (calc scalars) in framebuttons, put below in parent frame
        self.framebuttons = tk.Frame(self.listboxandbuttonsframe)
        self.framebuttons.pack(side=tk.BOTTOM,fill=tk.BOTH)
        self.framebutton1 = tk.Frame(self.framebuttons)
        self.framebutton1.pack(side=tk.TOP, fill= tk.X, expand=True)       
        self.buttonadd = tk.Button(self.framebutton1,
                                   text='Add '+self.dummy.ID +' Trace',
                                   command=self.addtrace)
        self.buttonadd.pack(side=tk.LEFT, fill=tk.X, expand= True)
        self.buttonremove = tk.Button(self.framebutton1,
                                      text='Remove '+ self.dummy.ID + ' Trace',
                                      command=self.removetrace)
        self.buttonremove.pack(side=tk.RIGHT, fill=tk.X, expand= True) 
        self.framebutton2 = tk.Frame(self.framebuttons)
        self.framebutton2.pack(side=tk.BOTTOM, fill= tk.X, expand=True)        
        self.buttoncalcstats = tk.Button(self.framebutton2,
                                         text='Calculate '+ self.dummy.ID + ' Statistics',
                                         command=self.plotstats)
        self.buttoncalcstats.pack(fill=tk.X, expand= True)

        #create frame instructions + label
        self.frameinstructions = tk.Frame(self.window, highlightbackground="grey",highlightthickness=4)
        tk.Label(self.frameinstructions,
                 text='1. Select files in list to display on left graph\n'+
                 '\n'+
                 '2. Click "Add/Remove '+self.dummy.ID+' Trace" Button to add/remove\n'+
                 'plots on left plot to right plot\n'+
                 '\n'+
                 '3. Select tick boxes to include in stats workup (max 10)\n'+
                 '\n'+
                 '4. Select independent variable below for stat v stat plots\n'+
                 '\n'+
                 '5. Click "Calculate '+self.dummy.ID+' Statistics" to calculate stats',
                 bg='white',
                 font=('Arial', self.fontsizesmall, 'bold')).pack(side=tk.TOP,fill=tk.BOTH,expand=True)
        
        #create frame for dropdown box, dropdown box, ensure values centered, fill
        self.frameindependentvar = tk.Frame(self.window, highlightbackground="grey",highlightthickness=4)
        self.independentvar = tk.StringVar()
        independentvarchosen = ttk.Combobox(self.frameindependentvar, textvariable = self.independentvar, justify='center')
        independentvarchosen.option_add('*TCombobox*Listbox.Justify', 'center')      
        self.independentvaroptions = ['None']     
        for scalar in self.dummy.scalarlist:
            self.independentvaroptions.append(self.dummy.Name[scalar])
        independentvarchosen['value'] = self.independentvaroptions
        independentvarchosen.set('None')
        independentvarchosen.pack(fill=tk.BOTH,expand=True)

        # create framevectorplots for xy curves, add fig1 (selected) to left of fig2 (added)
        self.framevectorplots = tk.Frame(self.window, bg='grey', highlightbackground="grey", highlightthickness=4)
        
        # fig1
        self.fig1 = Figure(figsize =(self.f12width/200,self.f12height/200))
        self.a = self.fig1.add_subplot(111)
        self.a.plot(0, 0, color='red')
        self.p1canvas = FigureCanvasTkAgg(self.fig1, master=self.framevectorplots)
        self.p1canvas.get_tk_widget().pack(side=tk.LEFT,fill=tk.BOTH,expand=True)
        ll, bb, ww, hh = self.a.get_position().bounds
        self.a.set_position([ll+0.2*ww, bb+0.2*hh, ww*0.8, hh*0.8])

        # fig2
        self.fig2 = Figure(figsize=(self.f12width/200,self.f12height/200))
        self.b = self.fig2.add_subplot(111)
        self.b.plot(0, 0, color='red')
        self.p2canvas = FigureCanvasTkAgg(self.fig2, master=self.framevectorplots)
        self.p2canvas.get_tk_widget().pack(side=tk.RIGHT,fill=tk.BOTH,expand=True)
        ll, bb, ww, hh = self.b.get_position().bounds
        self.b.set_position([ll+0.2*ww, bb+0.2*hh, ww*0.8, hh*0.8])
        
        #create dictionaries for checkboxes and checkbox variable
        self.vardict = {}
        self.cbdict = {}
        for scalar in self.dummy.scalarlist:
            self.vardict[scalar] = 'self.var'+ scalar 
            self.cbdict[scalar] = 'self.' + scalar + 'cb'
        
        # use dictionaries to pull scalars from class, create tick boxes spread across screen to decide what stats to plot
        self.framechecks = tk.Frame(self.window, bg='white')
        for scalar in self.dummy.scalarlist:
            self.vardict[scalar] = tk.IntVar(value=1)
            self.cbdict[scalar] = tk.Checkbutton(self.framechecks,
                                                text = self.dummy.Variable[scalar],
                                                variable = self.vardict[scalar]).pack(side=tk.LEFT, fill = tk.Y, expand=True)

        # add everything to window, because everything has been packed we can use .place with exact dimensions of object
        self.xloc = 0
        self.yloc = 0
        
        self.listboxandbuttonsframe.place(x=self.xloc, y=self.yloc,width = self.scrollwidth, height = self.scrollheight)
        self.xloc += self.scrollwidth + self.spacerwidth
        
        self.frameinstructions.place(x=self.xloc, y=self.yloc,width = self.f12width, height = self.f12height*4/5)
        self.yloc += self.f12height*4/5 + self.spacerheight       
        self.frameindependentvar.place(x=self.xloc, y=self.yloc,width = self.f12width, height = self.f12height*1/5-self.spacerheight)
        self.yloc -= (self.f12height*4/5 + self.spacerheight)
       
        self.xloc += self.f12width + self.spacerwidth
        
        self.framevectorplots.place(x=self.xloc, y=self.yloc, width=self.f12width*2, height=self.f12height)
        self.yloc += self.f12height + self.spacerheight
        self.xloc -= (self.f12width + self.spacerwidth)
        
        self.framechecks.place(x=self.xloc,y=self.yloc, width=self.statswidth*10, height = 20) 
        self.yloc+= 20 + self.spacerheight 
        
        self.framestats1 = tk.Frame(self.window, highlightbackground="grey",highlightthickness=1)
        self.framestats1.place(x=self.xloc,y=self.yloc, width=self.statswidth*10, height= 25)
        self.yloc+=25
        self.framestats2 = tk.Frame(self.window, highlightbackground="grey",highlightthickness=1)
        self.framestats2.place(x=self.xloc,y=self.yloc, width=self.statswidth*10, height=self.statsheight)
        self.yloc += self.statsheight + self.spacerheight
    
        self.framesvs1 = tk.Frame(self.window, highlightbackground="grey",highlightthickness=1)
        self.framesvs1.place(x=self.xloc,y=self.yloc, width=self.svswidth*5, height= self.svsheight)
        self.yloc+=self.svsheight +self.spacerheight
        self.framesvs2 = tk.Frame(self.window, highlightbackground="grey",highlightthickness=1)
        self.framesvs2.place(x=self.xloc,y=self.yloc, width=self.svswidth*5, height= self.svsheight)
        

    # when listbox item is clicked plots it on plot #1 and changes selected color item as legend
    def onselect(self, evt):

        # set w to the event selection, get index of selection and value
        w = evt.widget
        
        # clear graph a, load files, calc vectors, plot with colors & change color of listbox select to be legend
        self.a.clear()
        for z, values in enumerate(w.curselection()):
            value = w.get(values)
            c = self.cmap(float(z) / len(w.curselection()))
            self.filestoclass[str(value)] = self.classname()
            self.filestoclass[str(value)].loadfile(self.loadfolder+value)
            self.filestoclass[str(value)].calcvectors()
            self.a.plot(getattr(self.filestoclass[value], self.xydict['x']),
                       getattr(self.filestoclass[value], self.xydict['y']),
                       label = value,
                       color=c)
    
            self.listbox.itemconfig(self.listoffiles.index(str(value)),
                                    selectforeground=matplotlib.colors.to_hex(c,keep_alpha=False))
        
        # customize graph
        self.a.set_title("Selected "+ self.dummy.ID +" Waves", fontsize=self.fontsizelarge)
        self.a.annotate('See Highlighted Colors',
                        xy=(0.05,0.05),
                        xycoords='axes fraction',
                        horizontalalignment='left',
                        xytext=(0, 0),
                        textcoords='offset pixels',
                        verticalalignment='bottom',).set_bbox(dict(facecolor='white', alpha=1, edgecolor='black'))
        self.customizegraphs(self.a)

        # update
        self.p1canvas.draw()

        
    # manages add JV Button: adds JVs on plot #1 to self.analyzedwaves list, passes to updateplot2 to plot
    def addtrace(self):
       
        # cycle through and try to remove curves from list of waves to plot, then add
        for i in self.listbox.curselection():
            value = self.listbox.get(i)
            try:
                self.analyzedwaves.remove(value)
            except ValueError:
                pass
            self.analyzedwaves.append(value)

        # update plot #2
        self.updateplot2(self.analyzedwaves)

        
    # Manages Remove Button: removes XYs on plot #1 from self.analyzedwaves list, passes to updateplot2 to plot
    def removetrace(self):

        # cycle through and try to remove curves from plot
        for i in self.listbox.curselection():
            value = self.listbox.get(i)
            try:
                self.analyzedwaves.remove(value)
            except ValueError:
                pass

        # update plot #2
        self.updateplot2(self.analyzedwaves)

        
    # updates plot2, colors listbox for key
    def updateplot2(self, waves):

        # make listbox black
        for z in range(len(self.listoffiles)):
            self.listbox.itemconfig(z, foreground='black')

        # clear graph, add jv waves, plot with colors & change color of listbox text to be legend
        self.b.clear()
        for z, value in enumerate(waves):
            c = self.cmap(float(z) / len(waves))
            self.b.plot(getattr(self.filestoclass[value], self.xydict['x']),
                       getattr(self.filestoclass[value], self.xydict['y']),
                       label = value,
                       color=c)
            self.listbox.itemconfig(self.listoffiles.index(str(value)),
                                    foreground=matplotlib.colors.to_hex(c,keep_alpha=False))

        # customize graph
        self.b.set_title(self.dummy.ID + ' Waves for Statistics', fontsize=self.fontsizelarge)
        self.b.annotate('See Text Colors',
                        xy=(0.05,0.05),
                        xycoords='axes fraction',
                        horizontalalignment='left',
                        xytext=(0, 0),
                        textcoords='offset pixels',
                        verticalalignment='bottom',).set_bbox(dict(facecolor='white', alpha=1, edgecolor='black'))
        self.customizegraphs(self.b)
        
        # update
        self.p2canvas.draw()


    # manages ticks to determine what stats to plot
    def setparams(self):
        
        # cycle through list of scalars, if checkbox is ticked add to returnstring (manages stats to compile)
        returnstring = []
        for scalar in self.dummy.scalarlist:
            # creates variable for tickbox (to se if its checked), e.g. self.varjsc
            buttoncheck = self.vardict[scalar]
            if buttoncheck.get() == 1:
                returnstring.append(scalar)

        return returnstring

    
    # for each wave on plot 2, sets file to Class, uses class functions to calc scalars, plots
    def plotstats(self):

        # try to destroy values in the frames incase they filled with data        
        try:
            for widget in self.framestats1.winfo_children():
                widget.destroy()
        except AttributeError:
            pass
        try:
            for widget in self.framestats2.winfo_children():
                widget.destroy()
        except AttributeError:
            pass

        # create empty list vals_unzipped to populate with stats and set listofstats to checked boxes
        # HERE HERE HERE --> we could move towards numpy array to make faster.
        vals_unzipped = [0]*len(self.analyzedwaves)
        listofstats = self.setparams()

#NEW FIX HERE
        
        
        
        
# FIX HERE: edit this to not take return

        # only recalc scalars if need be, make list of them
        for zz, wave in enumerate(self.analyzedwaves): 
            if self.filestoclass[str(wave)].scalar == 0:
                self.filestoclass[str(wave)].calcscalars()
            vals_unzipped[zz] = self.filestoclass[str(wave)].getparamvals(listofstats) 

        # alter index so that instead of [[a1->an],[b1->bn]] we have 1's, 2's, n's grouped -- these are stats e.g. jsc
        vals = np.array(list(map(list, zip(*vals_unzipped))), dtype=np.float64)

        # cycle through parameters to plot
        for i in range(len(listofstats)):

            # create figure and plot axes
            fig3 = Figure(figsize=(self.statswidth/110, self.statsheight/440))
            a = fig3.add_subplot(111)
            l, b, w, h = a.get_position().bounds
            a.set_position([l+0.6*w, b, w*0.4, h])
            val = vals[i]
            
            # add boxplot
            sns.boxplot(y=val, ax=a, palette="pastel", showfliers=False)

            # append indivisual scans in scatter plot using same color as plot 2
            for ii in range(len(val)):
                c = self.cmap(float(ii) / len(val))
                sns.scatterplot(x=np.array([0]),
                                y=val[ii],
                                ax=a,
                                x_jitter=1,
                                color=c)

            # adjust ticks
            a.yaxis.set_major_formatter(ScalarFormatter())
            a.yaxis.set_major_locator(LinearLocator(numticks=5))
            a.yaxis.set_major_formatter(FormatStrFormatter('%1.3g'))
            a.tick_params(
                axis='x',  # changes apply to the x-axis
                which='both',  # both major and minor ticks are affected
                bottom=False,  # ticks along the bottom edge are off
                top=False,  # ticks along the top edge are off
                labelbottom=False)  # labels along the bottom edge are off
            
            # pack canvas to frame left to right
            canvas = FigureCanvasTkAgg(fig3, master=self.framestats2)
            canvas.get_tk_widget().pack(side=tk.LEFT, fill=tk.Y,expand=True,padx=1)
            canvas.draw()                

            # create labels for the plots, pack left to right as well
            tk.Label(self.framestats1, text=str(self.dummy.Label[listofstats[i]]),
                     bg='white',
                     width=int(self.statswidth/110),
                     font=('Arial', 10, 'bold', 'underline')).pack(side=tk.LEFT,fill=tk.BOTH,expand=True,padx=3)
    
        # just for now we will activate plotstatvstat hereherehere
        if self.independentvar.get() != 'None':
            self.plotstatvstat()

        
    # plots stat versus stat (scalar v scalar)
    def plotstatvstat(self):
        
        # gets the independent variable given the elaborated name
        independentvariable = self.dummy.NameRev[self.independentvar.get()]
        
        # try to destroy values in the frames incase they are filled with data        
        try:
            for widget in self.framesvs1.winfo_children():
                widget.destroy()
        except AttributeError:
            pass
        try:
            for widget in self.framesvs2.winfo_children():
                widget.destroy()
        except AttributeError:
            pass
               
        # get list of stats, create graph number to limit graphs per row/frame to 5, and set initial frame to framesvs1
        listofstats = self.setparams() 
        graphnumber = 0 
        targetframe = self.framesvs1
        
        # cycle through list of stats
        for i, stat in enumerate(listofstats): 
            
            # if we have 5 plots on first frame, move to the 2nd frame (framesvs2) for plotting.
            if(graphnumber >= 5):
                targetframe = self.framesvs2
                graphnumber -= 5
            
            # build figure, add plot, redimension
            fig4 = Figure(figsize=(self.svswidth/110,self.svsheight/440))
            compgraph = fig4.add_subplot(111)
            l, b, w, h = compgraph.get_position().bounds
            compgraph.set_position([l + 0.35 * w, b + 0.35 * h, w * 0.65, h * 0.65])

            # cycle through waves where stats have been calculated and plot all stats in listofstats v dependent variable
            for ii in range(len(self.analyzedwaves)):
                c = self.cmap(float(ii) / len(self.analyzedwaves))
                sns.scatterplot(x=[getattr(self.filestoclass[str(self.analyzedwaves[ii])], independentvariable)],
                                y=[getattr(self.filestoclass[str(self.analyzedwaves[ii])], listofstats[i])],
                                ax=compgraph,
                                color=c)
                
                compgraph.set_ylabel(self.dummy.Label[str(listofstats[i])], fontsize=self.fontsizesmall)
                compgraph.set_xlabel(self.dummy.Label[str(independentvariable)], fontsize=self.fontsizesmall)
            
            # customize graph x and y axes
            compgraph.xaxis.set_major_formatter(ScalarFormatter())
            compgraph.xaxis.set_major_locator(LinearLocator(numticks=3))
            compgraph.xaxis.set_major_formatter(FormatStrFormatter('%1.3g')) 
            compgraph.yaxis.set_major_formatter(ScalarFormatter())
            compgraph.yaxis.set_major_locator(LinearLocator(numticks=4))
            compgraph.yaxis.set_major_formatter(FormatStrFormatter('%1.3g'))

            # add canvas to frame using target frame to ensure we have a max of 5 per row. 
            canvas = FigureCanvasTkAgg(fig4, master=targetframe)
            canvas.get_tk_widget().pack(side=tk.LEFT, fill=tk.Y, expand=True)
            canvas.draw()
            graphnumber +=1


# Run Code

In [3]:
if __name__ == '__main__':
    window = tk.Tk()
    xyGraphingWindow(window)
    window.mainloop()